# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=66efcbb73e9f9467cba08c3f268d4cecbb610365674770c2a6d7b043964c497c
  Stored in directory: /root/.cache/pip/wheels/2a/62/75/3d74209bfebb8805823ae74afa28653aa1ea76d8b5a9d741ff
Successfully built gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=73VwbTDHyIaaljSGNtG4CcYjqxsoqO&prompt=consent&token_usage=remote&access_type=offline&code_chal

# Tratamento

In [ ]:
# Import necessary libraries
import pandas as pd
import pandas_gbq

# Define the SQL query to select civil pensioner data from a BigQuery table.
query = """ SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao, DATA_INICIO_PENSAO as data_inicio_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2025_4_siape_pensionista_cadastro`
        """

# Execute the query and load the result into a pandas DataFrame.
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
# Display the initial DataFrame.
df

# Check for null values in the 'data_inicio_pensao' column.
pensoes = df['data_inicio_pensao'].isnull()
# Count the number of null and non-null values.
pensoes.value_counts()

# Drop rows with any missing values.
df = df.dropna() # 42 rows were dropped, equivalent to the total null values indicated above.
# Display the DataFrame after dropping nulls.
df

# Convert the 'data_inicio_pensao' column to datetime objects.
df['data_inicio_pensao'] = pd.to_datetime(df['data_inicio_pensao'], dayfirst=True, format= "%d/%m/%Y") #transforming into date

# Extract the year from the 'data_inicio_pensao' column.
df['ano_inicio_pensao'] = df['data_inicio_pensao'].dt.year #getting only the year

# Define bin edges for creating decades.
limites = [0, 1929, 1939, 1949, 1959, 1969, 1979, 1989, 1999, 2009, 2019, 2025]
# Define labels for the decade bins.
categorias = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]

# Create a new column by categorizing the 'ano_inicio_pensao' into decades.
df['decada_inicio_pensao'] = pd.cut(df['ano_inicio_pensao'], bins=limites, labels=categorias)

# Convert the new 'decada_inicio_pensao' column to an integer type.
df['decada_inicio_pensao'] = df['decada_inicio_pensao'].astype(int)

# Select and display rows where 'tipo_pensao' is 'Inválido' for inspection.
df[df['tipo_pensao']=='Inválido'] # Corresponds to 0.58% of the total

# Drop the rows where the 'tipo_pensao' column is 'Inválido'.
df = df.drop(df.index[df['tipo_pensao']=='Inválido'])

# Define a dictionary to map detailed pension types to broader categories.
pensoes = {'(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a)':'Cônjuge ou ex-cônjuge',
'Filho (a) / Menor sob guarda / Enteado (a)':"Filhos, netos, enteados ou menores em tutela",
'Pai / Mae':'Pais',
'Outros':'Outros',
'Pessoa designada':'Pessoa designada',
'Irmao (a)':'Irmão (ã)',
'Neto (a)':'Filhos, netos, enteados ou menores em tutela"'}

# Define a function to apply the categorization using the dictionary.
def categorizando(x):
    if x in pensoes:
        return pensoes[x]

# Create a new column 'tipo_pensao_agrupado' by applying the function.
df['tipo_pensao_agrupado'] = df['tipo_pensao'].apply(categorizando)

# Select and reorder columns for the final DataFrame.
df= df[['ano', 'mes', 'data_inicio_pensao', 'ano_inicio_pensao', 'decada_inicio_pensao', 'tipo_pensao','tipo_pensao_agrupado', 'total']]
# Display the final DataFrame.
df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 276173 entries, 0 to 278632
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ano                   276173 non-null  Int64         
 1   mes                   276173 non-null  Int64         
 2   data_inicio_pensao    276173 non-null  datetime64[ns]
 3   ano_inicio_pensao     276173 non-null  int32         
 4   decada_inicio_pensao  276173 non-null  int64         
 5   tipo_pensao           276173 non-null  object        
 6   tipo_pensao_agrupado  276173 non-null  object        
 7   total                 276173 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 18.7+ MB


# Upload

In [ ]:
# Import the bigquery library from google.cloud
from google.cloud import bigquery

# Initialize the BigQuery client, specifying the Google Cloud project ID.
# This client object is the main entry point for interacting with the BigQuery API.
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create a reference to the BigQuery dataset named 'perfil_remuneracao'.
# This object points to the dataset where the table will be created or updated.
dataset_ref = client.dataset('perfil_remuneracao')

# Define the schema for the destination BigQuery table.
# The schema is a list of SchemaField objects, where each object defines a column's:
# 1. Name (e.g., 'ano')
# 2. Data type (e.g., 'INTEGER')
# 3. Description (e.g., 'Ano de referência da observação')
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),
        bigquery.SchemaField('data_inicio_pensao','DATETIME',description='Data de início da pensão'),
        bigquery.SchemaField('ano_inicio_pensao','INTEGER',description='Ano de início da pensão extraido da data'),
        bigquery.SchemaField('decada_inicio_pensao','INTEGER',description='Década de início da pensão'),
        bigquery.SchemaField('tipo_pensao','STRING',description='Tipo de pensão'),
        bigquery.SchemaField('tipo_pensao_agrupado','STRING',description='Categorização das pensões'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')
        ]

# Create a reference to the target table within the dataset specified earlier.
# The table will be named 'SIAPE_inicio_pensoes_total_v8'.
table_ref = dataset_ref.table('SIAPE_inicio_pensoes_total_v8')

# Configure the load job by creating a LoadJobConfig object.
# Here, we specify the schema that BigQuery should use for the table. This ensures
# that the columns in BigQuery have the correct data types and descriptions.
job_config = bigquery.LoadJobConfig(schema=schema)

# Start the job to load data from the pandas DataFrame 'df' into the specified BigQuery table ('table_ref').
# The job is configured with the previously defined 'job_config'. This command sends the data to BigQuery.
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the load job to complete and retrieve its result.
# This line is blocking and will pause the script's execution until the data upload is finished.
# It's crucial for ensuring the data is fully loaded before the script ends or proceeds.
job.result()